### Pasos a seguir

1. Bajar resolucion de imagenes para reducir costos (Python)
2. Orgnizar imagenes (Python)
3. Pasar imagenes a asistente para clasificar (OpenAI)
    - Pedir también el centro de la cara como un punto dentro de la imagen para obtener el "centro" de la imagen para centrar recortar posterior a la detección
4. Ordenar imagenes (Python)
    - Definir una funcion para centrar el rostro en python (este paso lo dejamos como mejoras a futuro)
5. Crear documento (Python)

Final --- Juntar todo

### Precios de imágenes:
https://openai.com/api/pricing/

Asistentes 
 - https://platform.openai.com/assistants/asst_lFBaNBuv7qFsLzIoeKKh7vpY
 

Para un asistente de clasificación de imágenes como “Detección de Rostros”, conviene hacerlo lo más determinista posible, así que sí, bajar tanto la temperature como el top_p ayuda a que las respuestas sean consistentes y sin variaciones innecesarias.

temperature: ponla en un valor muy bajo, por ejemplo 0 o 0.1. Así el modelo elegirá siempre la opción más probable.

top_p: también puedes reducirla a alrededor de 0.8 o incluso 0.5 si quieres acotar aún más las posibles salidas. Si la dejas en 1.0, funcionará bien, pero con valores menores restringes el rango de tokens considerados.

# Escalar imagenes

In [1]:
from PIL import Image, ImageOps
import os
import glob

SRC_DIR = "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/img/jose/frente"
DST_DIR = "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/img/jose/frente_resized"
os.makedirs(DST_DIR, exist_ok=True)

TARGET_WIDTH = 800

def resize_image(src_path, dst_path, target_width):
    img = Image.open(src_path)
    # Corrige la orientación según EXIF
    img = ImageOps.exif_transpose(img)
    # Calcula la nueva altura manteniendo proporción
    w_percent = (target_width / img.width)
    target_height = int(img.height * w_percent)
    # Redimensiona
    img_resized = img.resize((target_width, target_height), Image.LANCZOS)
    # Guarda (por defecto mantiene formato JPEG, PNG, etc.)
    img_resized.save(dst_path)
    img.close()

for img_path in glob.glob(os.path.join(SRC_DIR, "*.*")):
    filename = os.path.basename(img_path)
    dst_path = os.path.join(DST_DIR, filename)
    resize_image(img_path, dst_path, TARGET_WIDTH)
    print(f"Guardada: {dst_path}")


Guardada: /Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/img/jose/frente_resized/JOSE_MARIO_MORENO_ZAVALA_7530aaa3838b3135fb60e1df0ceb8fee.JPG
Guardada: /Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/img/jose/frente_resized/JOSE_MARIO_MORENO_ZAVALA_3ad2f98e1075b76bfa9e69963f957385.JPG
Guardada: /Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/img/jose/frente_resized/JOSE_MARIO_MORENO_ZAVALA_2ee77858f2bb3f0f2effa4717ba33863.JPG
Guardada: /Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/img/jose/frente_resized/JOSE_MARIO_MORENO_ZAVALA_5a9e480a8788dd5a5dae2025e25e475d.JPG
Guardada: /Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/img/jose/frente_resized/JOSE_MARIO_MORENO_ZAVALA_e5c8253b917c53ff27a9ea8625d6a845.JPG
Guardada: /Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/img/jose/frente_resized/JOSE_MARIO_MORENO_ZAVALA_5d7e0fe35521b61f82c2164788e822f1.JPG
Guardada: /Users/aaron/Documentos/INFOTE

# API ENV

In [5]:
import os
import time
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

# Inicializa el cliente

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.getenv("OPENAI_API_KEY"),
)
OPENAI_ASSISTANT_ID = os.getenv("OPENAI_ASSISTANT_ID") 

In [6]:
prompt = """
Recibe un lote de imágenes del mismo sujeto y realiza dos tareas por imagen:

1 - Clasificación facial:
Posición de la cabeza: Elige una sola opción entre las siguientes:
- Perfil completo a la izquierda
- Perfil completo a la derecha
- Ligeramente girado a la izquierda
- Ligeramente girado a la derecha
- Frente

Expresión facial: Elige una sola opción entre las siguientes:
- Sin mostrar dentadura (serio o neutro)
- Mostrando dentadura (sonrisa abierta)

2 - Detección de puntos clave:
Identifica las coordenadas (x, y) en píxeles de al menos los siguientes puntos anatómicos visibles:
- Punta de la nariz
- Centro de la boca (entre los labios)
- Centro de la frente (aproximadamente entre las cejas)

Manejo de imágenes borrosas o ambiguas:
Si la imagen está demasiado borrosa o la cabeza está en una posición ambigua, escribe "indefinida" tanto en posición como en expresión.
También indica "puntos no detectados" si no puedes identificar de forma confiable las coordenadas.

Formato de salida:

Por cada imagen válida, genera una línea con el siguiente formato:
nombre_del_archivo: posición – <posición>, expresión – <expresión>, nariz – (x, y), boca – (x, y), frente – (x, y)

Además, necesito un total de 10 imágenes: una por cada combinación de posición de cabeza y expresión facial (5 posiciones × 2 expresiones).
Solo selecciona las 10 mejores imágenes que representen claramente cada categoría.
Si hay imágenes duplicadas, borrosas o de menor calidad comparadas con otras, indícalo explícitamente y menciona que serán descartadas o ignoradas por esas razones.

"""

# Tests

In [12]:
def run_assistant(thread):
    # Retrieve the Assistant
    assistant = client.beta.assistants.retrieve(OPENAI_ASSISTANT_ID)

    # Run the assistant
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
        # instructions=f"You are having a conversation with {name}",
    )

    # Wait for completion
    # https://platform.openai.com/docs/assistants/how-it-works/runs-and-run-steps#:~:text=under%20failed_at.-,Polling%20for%20updates,-In%20order%20to
    while run.status != "completed":
        # Be nice to the API
        time.sleep(0.5)
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

    # Retrieve the Messages
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    new_message = messages.data[0].content[0].text.value
    return new_message

def generate_response(message_body):
    # Check if there is already a thread_id for the wa_id
    thread_id = None

    # If a thread doesn't exist, create one and store it
    if thread_id is None:
        thread = client.beta.threads.create()
        thread_id = thread.id

    # Otherwise, retrieve the existing thread
    else:
        thread = client.beta.threads.retrieve(thread_id)

    # Add message to thread
    message = client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=message_body,
    )

    # Run the assistant and get the new message
    new_message = run_assistant(thread)

    return new_message

In [13]:
message_body = "Hola, ¿me escuchas?"

In [14]:
response = generate_response(message_body)

In [15]:
response

'¡Hola! Sí, te escucho. ¿En qué puedo ayudarte hoy?'

In [44]:
import base64
from openai import OpenAI

img_path = "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/img/jose/frente_resized/JOSE_MARIO_MORENO_ZAVALA_00b880189f5efe4752ea19c4ac1feaa3.JPG"
with open(img_path, "rb") as image_file:
    b64_image = base64.b64encode(image_file.read()).decode("utf-8")

client = OpenAI()

# prompt = "What is in this image?"

response = client.responses.create(
    model="o4-mini",
    input=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": prompt},
                {"type": "input_image", "image_url": f"data:image/png;base64,{b64_image}"},
            ],
        }
    ],
)

In [45]:
print(response.output[1].content[0].text)

imagen1.jpg: posición – ligeramente girado a la derecha, expresión – mostrando dentadura


In [46]:
import base64
from openai import OpenAI

img_path = "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/img/jose/frente_resized/JOSE_MARIO_MORENO_ZAVALA_3ad2f98e1075b76bfa9e69963f957385.JPG"
with open(img_path, "rb") as image_file:
    b64_image = base64.b64encode(image_file.read()).decode("utf-8")

client = OpenAI()

# prompt = "What is in this image?"

response = client.responses.create(
    model="o4-mini",
    input=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": prompt},
                {"type": "input_image", "image_url": f"data:image/png;base64,{b64_image}"},
            ],
        }
    ],
)

In [47]:
print(response.output[1].content[0].text)

imagen1.jpg: posición – perfil completo a la derecha, expresión – sin mostrar dentadura


# Tests 2

In [6]:
import base64
from openai import OpenAI

img_path = "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/img/jose/frente_resized/JOSE_MARIO_MORENO_ZAVALA_00b880189f5efe4752ea19c4ac1feaa3.JPG"
with open(img_path, "rb") as image_file:
    b64_image = base64.b64encode(image_file.read()).decode("utf-8")

client = OpenAI()

# prompt = "What is in this image?"

response = client.responses.create(
    model="o4-mini",
    input=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": prompt},
                {"type": "input_image", "image_url": f"data:image/png;base64,{b64_image}"},
            ],
        }
    ],
)

In [7]:
response

Response(id='resp_68277d4292e88191b91cbbe5f324c1450b572fd42b0a69ca', created_at=1747418434.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='o4-mini-2025-04-16', object='response', output=[ResponseReasoningItem(id='rs_68277d43645081918adfabbfa58beafb0b572fd42b0a69ca', summary=[], type='reasoning', encrypted_content=None, status=None), ResponseOutputMessage(id='msg_68277d4b7b6881918778e2b53922ace40b572fd42b0a69ca', content=[ResponseOutputText(annotations=[], text='imagen1.jpg: posición – ligeramente girado a la derecha, expresión – mostrando dentadura', type='output_text')], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, max_output_tokens=None, previous_response_id=None, reasoning=Reasoning(effort='medium', generate_summary=None, summary=None), service_tier='default', status='completed', text=ResponseTextConfig(format=ResponseFormatText(type='text')), truncation='dis

In [7]:
import base64
from openai import OpenAI

def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# Rutas de las imágenes
img_path_1 = "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/img/jose/frente_resized/JOSE_MARIO_MORENO_ZAVALA_00b880189f5efe4752ea19c4ac1feaa3.JPG"
img_path_2 = "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/img/jose/frente_resized/JOSE_MARIO_MORENO_ZAVALA_2ee77858f2bb3f0f2effa4717ba33863.JPG"

# Codificación en base64
b64_image_1 = encode_image_to_base64(img_path_1)
b64_image_2 = encode_image_to_base64(img_path_2)


In [8]:

client = OpenAI()

# prompt = "What is in this image?"

response = client.responses.create(
    model="o4-mini",
    input=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": prompt},
                {"type": "input_image", "image_url": f"data:image/png;base64,{b64_image_1}"},
                {"type": "input_image", "image_url": f"data:image/png;base64,{b64_image_2}"},
            ],
        }
    ],
)

In [11]:
print(response.output[1].content[0].text)

foto1.jpg: posición – ligeramente girado a la derecha, expresión – mostrando dentadura  
foto2.jpg: posición – ligeramente girado a la izquierda, expresión – mostrando dentadura


# Ordenar en doc

In [14]:
from reportlab.lib.pagesizes import A4, landscape
from reportlab.pdfgen import canvas
import os

def images_to_pdf(img_paths, output_path):
    # Página en horizontal (A4 landscape)
    width, height = landscape(A4)
    c = canvas.Canvas(output_path, pagesize=(width, height))

    # Márgenes y espacio
    margin = 40
    usable_width = width / 2 - 2 * margin   # ancho de cada columna
    usable_height = (height - 3 * margin) / 2  # alto de cada fila (2 filas)

    # Coordenadas para la columna izquierda
    x = margin
    y_top = height - margin - usable_height
    y_bottom = margin

    # Para cada imagen, calculamos escala manteniendo ratio
    def draw_image(path, x, y):
        img_width, img_height = c.drawImage(path, x, y, width=0, height=0, preserveAspectRatio=True, mask='auto')
        # Si la imagen excede el espacio asignado, la escalamos
        scale = min(usable_width / img_width, usable_height / img_height, 1)
        iw, ih = img_width * scale, img_height * scale
        c.drawImage(path, x, y + (usable_height - ih), width=iw, height=ih, preserveAspectRatio=True, mask='auto')

    # Dibuja la primera imagen arriba
    draw_image(img_paths[0], x, y_top)

    # Dibuja la segunda imagen abajo
    draw_image(img_paths[1], x, y_bottom)

    c.showPage()
    c.save()

if __name__ == "__main__":
    imgs = [
        "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/img/jose/frente_resized/JOSE_MARIO_MORENO_ZAVALA_00b880189f5efe4752ea19c4ac1feaa3.JPG",
        "/Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/img/jose/frente_resized/JOSE_MARIO_MORENO_ZAVALA_2ee77858f2bb3f0f2effa4717ba33863.JPG",
    ]
    out_pdf = "comparacion_jose_landscape.pdf"
    images_to_pdf(imgs, out_pdf)
    print(f"PDF generado en: {os.path.abspath(out_pdf)}")


PDF generado en: /Users/aaron/Documentos/INFOTEC RETRO/python-cv-faceorientation/notebooks/comparacion_jose_landscape.pdf
